In [1]:
%pip install tensorflow==2.4.1 tensorflow-gpu==2.4.1 opencv-python mediapipe sklearn matplotlib

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement tensorflow==2.4.1 (from versions: 2.12.0rc0, 2.12.0rc1, 2.12.0, 2.12.1, 2.13.0rc0, 2.13.0rc1, 2.13.0rc2, 2.13.0, 2.13.1, 2.14.0rc0, 2.14.0rc1, 2.14.0, 2.14.1, 2.15.0rc0, 2.15.0rc1, 2.15.0, 2.15.1, 2.16.0rc0, 2.16.1)
ERROR: No matching distribution found for tensorflow==2.4.1

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [2]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose


In [3]:
def mediapipe_detection(image,model):
    image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    result = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)
    return image,result

In [4]:
def draw_landmarks(image,results):
    mp_drawing.draw_landmarks(image,results.face_landmarks,mp_holistic.FACEMESH_TESSELATION)
    mp_drawing.draw_landmarks(image,results.pose_landmarks,mp_pose.POSE_CONNECTIONS)
    mp_drawing.draw_landmarks(image,results.left_hand_landmarks,mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(image,results.right_hand_landmarks,mp_holistic.HAND_CONNECTIONS)

In [5]:
def draw_styled_landmarks(image,results):
    mp_drawing.draw_landmarks(image,results.face_landmarks,mp_holistic.FACEMESH_TESSELATION,mp_drawing.DrawingSpec(color=(80,110,10),thickness=1,circle_radius=1),mp_drawing.DrawingSpec(color=(80,256,121),thickness=1,circle_radius=1))
    mp_drawing.draw_landmarks(image,results.pose_landmarks,mp_pose.POSE_CONNECTIONS,mp_drawing.DrawingSpec(color=(80,110,10),thickness=2,circle_radius=4),mp_drawing.DrawingSpec(color=(80,44,121),thickness=2,circle_radius=2))
    mp_drawing.draw_landmarks(image,results.left_hand_landmarks,mp_holistic.HAND_CONNECTIONS,mp_drawing.DrawingSpec(color=(121,122,76),thickness=2,circle_radius=4),mp_drawing.DrawingSpec(color=(121,44,76),thickness=2,circle_radius=4))
    mp_drawing.draw_landmarks(image,results.right_hand_landmarks,mp_holistic.HAND_CONNECTIONS,mp_drawing.DrawingSpec(color=(245,117,66),thickness=2,circle_radius=4),mp_drawing.DrawingSpec(color=(245,66,230),thickness=2,circle_radius=2))

In [6]:
def extract_keypoints(result):
    pose = np.array([[res.x,res.y,res.z,res.visibility]for res in result.pose_landmarks.landmark]).flatten() if result.left_hand_landmarks else np.zeros(33*4)
    lh = np.array([[res.x,res.y,res.z] for res in result.left_hand_landmarks.landmark]).flatten() if result.left_hand_landmarks else np.zeros(21*3)
    face = np.array([[res.x,res.y,res.z] for res in result.face_landmarks.landmark]).flatten() if result.face_landmarks else np.zeros(1404)
    rh = np.array([[res.x,res.y,res.z] for res in result.right_hand_landmarks.landmark]).flatten() if result.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose,lh,face,rh])

In [15]:
DATA_PATH = os.path.join('MP_DATA')
print(DATA_PATH)
actions = np.array(['hello','thanks','iloveyou'])
no_sequence = 30
sequence_length = 30

MP_DATA


In [13]:
for action in actions:
    for sequence in range(no_sequence):
        try:
            path = os.path.join(DATA_PATH,action,str(sequence))
            print(path)
            os.makedirs(os.path.join(DATA_PATH,action,str(sequence)))
        except:
            pass
            

M_DATA\hello\0
M_DATA\hello\1
M_DATA\hello\2
M_DATA\hello\3
M_DATA\hello\4
M_DATA\hello\5
M_DATA\hello\6
M_DATA\hello\7
M_DATA\hello\8
M_DATA\hello\9
M_DATA\hello\10
M_DATA\hello\11
M_DATA\hello\12
M_DATA\hello\13
M_DATA\hello\14
M_DATA\hello\15
M_DATA\hello\16
M_DATA\hello\17
M_DATA\hello\18
M_DATA\hello\19
M_DATA\hello\20
M_DATA\hello\21
M_DATA\hello\22
M_DATA\hello\23
M_DATA\hello\24
M_DATA\hello\25
M_DATA\hello\26
M_DATA\hello\27
M_DATA\hello\28
M_DATA\hello\29
M_DATA\thanks\0
M_DATA\thanks\1
M_DATA\thanks\2
M_DATA\thanks\3
M_DATA\thanks\4
M_DATA\thanks\5
M_DATA\thanks\6
M_DATA\thanks\7
M_DATA\thanks\8
M_DATA\thanks\9
M_DATA\thanks\10
M_DATA\thanks\11
M_DATA\thanks\12
M_DATA\thanks\13
M_DATA\thanks\14
M_DATA\thanks\15
M_DATA\thanks\16
M_DATA\thanks\17
M_DATA\thanks\18
M_DATA\thanks\19
M_DATA\thanks\20
M_DATA\thanks\21
M_DATA\thanks\22
M_DATA\thanks\23
M_DATA\thanks\24
M_DATA\thanks\25
M_DATA\thanks\26
M_DATA\thanks\27
M_DATA\thanks\28
M_DATA\thanks\29
M_DATA\iloveyou\0
M_DATA\ilove

In [14]:
cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic:
    for action in actions:
        for sequence in range(no_sequence):
            for frame_num in range(sequence_length):
                ret,frame = cap.read()
                image,result = mediapipe_detection(frame,holistic)
                draw_styled_landmarks(image,result)
                if frame_num == 0:
                    cv2.putText(image,'STARTING COLLECTION',(120,200),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,255),4,cv2.LINE_AA)
                    cv2.putText(image,'Collecting frame for {} Video number {}'.format(action,sequence),(15,12),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,255),1,cv2.LINE_AA)
                    cv2.waitKey(2000)
                else:
                    cv2.putText(image,'Collecting frame for {} Video number {}'.format(action,sequence),(15,12),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,255),1,cv2.LINE_AA)

                cv2.imshow('Open CV feed',image)
                keypoints = extract_keypoints(result)
                npy_path = os.path.join(DATA_PATH,action,str(sequence),str(frame_num))
                np.save(npy_path,keypoints)
                if(cv2.waitKey(10) & 0xFF == ord('q')):
                    break
        
cap.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 

In [26]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [9]:
label_map = {label:num for num,label in enumerate(actions)}

In [10]:
label_map

{'hello': 0, 'thanks': 1, 'iloveyou': 2}

In [21]:
sequences,labels = [],[]
for action in actions:
    for sequence in range(no_sequence):
        window = []
        for frame_num in range(sequence_length):
            path = os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num))
            res = np.load(path)
            window.append(res);
        sequences.append(window)
        labels.append(label_map[action])

In [44]:
x=np.array(sequences)
y = to_categorical(labels).astype(int)
print(x.shape)
print(y.shape)
y

(90, 30, 1662)
(90, 3)


array([[1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0,

In [46]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.05)

In [49]:
print("")